# gridfinder
Run through the full gridfinder model from data input to final guess for Burundi.
Note that the 'truth' data used for the grid here is very bad, so the accuracy results don't mean much.

## Before running the notebook

Install the library and its dependencies with, if you haven't done so already
```
pip install -e .
```
from the root directory. You can also execute this command directly in the notebook but will need to reload the
kernel afterwards

In [ ]:
# Note - this cell should be executed only once per session
%load_ext autoreload
%autoreload 2

import sys, os

# in order to get the config, it is not part of the library
os.chdir("..")
sys.path.append(os.path.abspath("."))

In [ ]:

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

import numpy as np

import folium

import gridfinder as gf
from gridfinder import save_raster
from config import get_config

## Set folders and parameters

In [ ]:
c = get_config(reload=True)

folder_ntl_in = c.datafile_path("ntl", stage=c.RAW)
aoi_in = c.datafile_path("small_aoi.geojson", stage=c.GROUND_TRUTH)
roads_in = c.datafile_path("smaller_roads.geojson", stage=c.GROUND_TRUTH)
pop_in = c.datafile_path('pop.tif', stage=c.GROUND_TRUTH)
grid_truth = c.datafile_path('grid.gpkg', stage=c.GROUND_TRUTH)

folder_ntl_out = c.datafile_path('ntl_clipped', stage=c.PROCESSED, check_existence=False)
raster_merged_out = c.datafile_path( 'ntl_merged.tif', stage=c.PROCESSED, check_existence=False)
targets_out =  c.datafile_path('targets.tif', stage=c.PROCESSED, check_existence=False)
targets_clean_out = c.datafile_path('targets_clean.tif', stage=c.CLEANED, check_existence=False)
roads_out = c.datafile_path('roads.tif', stage=c.PROCESSED, check_existence=False)

dist_out = c.datafile_path('dist.tif', stage=c.PROCESSED, check_existence=False)
guess_out = c.datafile_path('guess.tif', stage=c.PROCESSED, check_existence=False)
guess_skeletonized_out = c.datafile_path('guess_skel.tif', stage=c.PROCESSED, check_existence=False)
guess_nulled = c.datafile_path('guess_nulled.tif', stage=c.PROCESSED, check_existence=False)
guess_vec_out = c.datafile_path('guess.gpkg', stage=c.PROCESSED, check_existence=False)
animate_out = os.path.join(c.visualizations, 'guess.tif')

In [ ]:
percentile = 70      # percentile value to use when merging monthly NTL rasters
ntl_threshold = 0.1  # threshold when converting filtered NTL to binary (probably shouldn't change)
upsample_by = 2      # factor by which to upsample before processing roads (both dimensions are scaled by this)
cutoff = 0.0         # cutoff to apply to output dist raster, values below this are considered grid

## Clip  and merge monthly rasters

In [ ]:
gf.clip_rasters(folder_ntl_in, folder_ntl_out, aoi_in)

In [ ]:
raster_merged, affine = gf.merge_rasters(folder_ntl_out, percentile=percentile)
save_raster(raster_merged_out, raster_merged, affine)
print('Merged')
plt.imshow(raster_merged, vmin=0, vmax=1)

## Create filter

In [ ]:
ntl_filter = gf.NightlightFilter()

X = np.fromfunction(lambda i, j: i, ntl_filter.filter.shape)
Y = np.fromfunction(lambda i, j: j, ntl_filter.filter.shape)

fig = plt.figure()
sns.set()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, ntl_filter.filter, cmap=cm.coolwarm, linewidth=0, antialiased=False)

## Clip, filter and resample NTL

In [ ]:
ntl_thresh, affine = gf.prepare_ntl(raster_merged_out,
                                    aoi_in,
                                    electrification_predictor=ntl_filter,
                                    threshold=ntl_threshold,
                                    upsample_by=upsample_by)
save_raster(targets_out, ntl_thresh, affine)
print('Targets prepared')
plt.imshow(ntl_thresh, cmap='viridis')

## Remove target areas with no underlying population

In [ ]:
targets_clean = gf.drop_zero_pop(targets_out, pop_in, aoi_in)
save_raster(targets_clean_out, targets_clean, affine)
print('Removed zero pop')
plt.imshow(ntl_thresh, cmap='viridis')

## Roads: assign values, clip and rasterize

In [ ]:
roads_raster, affine = gf.prepare_roads(roads_in,
                                        aoi_in,
                                        targets_out)
save_raster(roads_out, roads_raster, affine, nodata=-1)
print('Costs prepared')
plt.imshow(roads_raster, cmap='viridis', vmin=0, vmax=1)

## Get targets and costs and run algorithm

In [ ]:
targets, costs, start, affine = gf.get_targets_costs(targets_clean_out, roads_out)
est_mem = gf.estimate_mem_use(targets, costs)
print(f'Estimated memory usage: {est_mem:.2f} GB')

In [ ]:
dist = gf.optimise(targets, costs, start,
                   jupyter=True,
                   animate=True,
                   affine=affine,
                   animate_path=animate_out)
save_raster(dist_out, dist, affine)
plt.imshow(dist)

## Filter dist results to grid guess

In [ ]:
guess = gf.read_and_threshold_distances(dist_out, cutoff=cutoff)
save_raster(guess_out, guess, affine)
print('Got guess')
plt.imshow(guess, cmap='viridis')

## Check results

In [ ]:
# TODO: the ground truth is not adjusted to the smaller aoi, execution of this cell leads to a shape mismatch

# true_pos, false_neg = gf.accuracy(grid_truth, guess_out, aoi_in)
# print(f'Points identified as grid that are grid: {100*true_pos:.0f}%')
# print(f'Actual grid that was missed: {100*false_neg:.0f}%')

## Skeletonize

In [ ]:
guess_skel = gf.thin(guess_out)
save_raster(guess_skeletonized_out, guess_skel, affine)
print('Skeletonized')
plt.imshow(guess_skel)

## Convert to geometry

In [ ]:
guess_gdf = gf.raster_to_lines(guess_skeletonized_out)
guess_gdf.to_file(guess_vec_out, driver='GPKG')
print('Converted to geom')

In [ ]:
minx, miny, maxx, maxy = list(guess_gdf.bounds.iloc[0])
bounds = ((miny, minx), (maxy, maxx))

m = folium.Map(control_scale=True)
m.fit_bounds(bounds)
folium.GeoJson(guess_gdf).add_to(m)
m